In [2]:
import pandas as pd
import numpy as np

In [10]:
raw_df = pd.read_csv('nba_stats_dataset.csv')

columns_to_keep = [
    'PLAYER', 'GP', 'MIN', 'PTS', 
    'C&S\nPTS', 'C&S\nFG%', 
    'Pull Up\nPTS', 'Pull Up\nFG%', 
    'eFG%', 'Shot Quality', 
    'Assisted 2s\n PTS', 'Unassisted 2s\n PTS',
    'Assisted 3s\n PTS', 'Unassisted 3s\n PTS'
]

cleaned = raw_df[columns_to_keep].copy()

cleaned.columns = [
    'Player', 'GP', 'MIN', 'PPG',
    'C&S PTS', 'C&S FG%',
    'PU PTS', 'PU FG%',
    'eFG%', 'Shot Quality',
    'Assisted 2s', 'Unassisted 2s',
    'Assisted 3s', 'Unassisted 3s'
]

cleaned


,Player,GP,MIN,PPG,C&S PTS,C&S FG%,PU PTS,PU FG%,eFG%,Shot Quality,Assisted 2s,Unassisted 2s,Assisted 3s,Unassisted 3s
0,Amen Thompson,69,32.3,14.1,1.1,29.2,1.0,35.4,57.5,0.56,5.51,5.01,1.04,0.04
1,Andrew Nembhard,65,28.9,10.0,2.1,32.1,2.8,43.2,50.6,0.51,2.86,3.17,1.94,0.42
2,Anfernee Simons,70,32.7,19.3,5.0,37.7,6.9,38.3,52.1,0.51,1.40,6.14,5.61,3.60
3,Anthony Edwards,79,36.3,27.6,3.5,42.2,11.3,37.8,54.7,0.54,3.90,6.25,5.77,6.38
4,Austin Reaves,73,34.9,20.2,4.8,40.0,5.3,38.4,55.6,0.52,3.34,4.25,5.92,2.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,Tyler Herro,77,35.4,23.9,5.5,40.1,8.1,42.9,56.3,0.52,3.92,6.47,7.36,2.42
82,Tyrese Haliburton,73,33.6,18.6,2.6,42.4,9.1,41.3,58.2,0.52,2.77,4.30,4.73,4.23
83,Tyrese Maxey,52,37.7,26.3,3.5,35.5,8.7,37.1,51.1,0.54,3.73,8.42,4.56,4.73
84,Tyus Jones,81,26.8,10.2,5.4,43.5,2.2,39.4,57.0,0.52,1.36,2.10,5.37,0.78
